In [2]:
from numba import jit, vectorize, float32, float64
import numpy as np
from scipy import spatial, integrate, optimize
from matplotlib import pyplot as plt
from SmoothingLength import HsmlIter

@vectorize([float32(float32), float64(float64)])
def Kernel3D(q):
    if q <= 0.5:
        return 2.5464790894703255 * (1 - 6*q**2 + 6*q**3)
    elif q <= 1.0:
        return 2.5464790894703255 * 2 * (1-q)**3
    else: return 0.0
    
@vectorize([float32(float32), float64(float64)])
def DKernel3D(q):
    if q <= 0.5:
        return 2.5464790894703255 * (-12*q + 18*q**2)
    elif q <= 1.0:
        return -2.5464790894703255 * 6 * (1-q)**2
    else: return 0.0

In [2]:
%timeit Kernel3D(np.random.rand(1e6))

C:\Users\Pika\Anaconda2\lib\site-packages\ipykernel\__main__.py:257: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


10 loops, best of 3: 20.2 ms per loop


In [6]:
N=  1000
#x = np.random.rand(N,1)**0.5
#x = (np.arange(32)/32.)
#x = x + 0.001*np.sin(2*np.pi*x)
#y = np.arange(32)/32.
#x,y,z = np.meshgrid(x,y,y)
#x = np.c_[x.flatten(),y.flatten(),z.flatten()]
#x = (x)%1.0
dtmin = (5./3)**0.5 / N / 100
T = 1./(5./3)**0.5
nsteps = int(T/dtmin+1)
dt = T/nsteps
print dt
m = np.ones(N)/N
gamma = 5./3
x = np.array([np.arange(0,1000)/1000.,]).T 
x += 0.001*np.sin(2*np.pi*x)
x = x % 1.0
#tree = spatial.cKDTree(x,boxsize=1.0)
des_ngb = 8
v = np.zeros_like(x)

1.29097293252e-05


In [4]:
tree = spatial.cKDTree(x, boxsize=1.0)
ngbdist, ngb = tree.query(x,des_ngb)
h = HsmlIter(ngb,ngbdist, error_norm=1e-12,dim=1)
rho = m * des_ngb / (2*h)
q = ngbdist/h[:,np.newaxis]
K = Kernel3D(q)
dK = DKernel3D(q)
FI = (1 + h / (3*rho) * np.sum((-m[:,np.newaxis]*h[:,np.newaxis]**-4 * (dK *q + 3*K)),axis=1))**-1
F = HydroForce(x,m,FI, K, dK, rho, h, ngb, ngbdist)
v += F * dt
x += v * dt

NameError: name 'x' is not defined

In [ ]:
e = []
for i in xrange(1000):
    if i%100 == 0: print i
    tree = spatial.cKDTree(x, boxsize=1.0)
    ngbdist, ngb = tree.query(x,des_ngb)
    h = HsmlIter(ngb,ngbdist, error_norm=1e-12,dim=1)
    rho = m * des_ngb / (2*h)
    q = ngbdist/h[:,np.newaxis]
    K = Kernel3D(q)
    dK = DKernel3D(q)
    FI = (1 + h / (3*rho) * np.sum((-m[:,np.newaxis]*h[:,np.newaxis]**-4 * (dK *q + 3*K)),axis=1))**-1
    F = HydroForce(x,m,FI, K, dK, rho, h, ngb, ngbdist)
    v += F * dt
    x += v * dt
    x = x % 1.0
    
    e.append(np.sum(0.5*m*v**2) + np.sum(3./2 * rho**(5./3)))
plt.plot(e)
plt.yscale('log')
plt.show()
#if i % (nsteps/10)==0:
    #if i%10==0:
    #    plt.scatter(x[:,0],v[:,0])
    #
#plt.scatter(x[:,0][(x[:,1]==0.0)*(x[:,2]==0.0)], )
        #plt.show()

0
100
200
300
400
500
600
700
800
900


In [3]:
@jit
def HydroForce(x, m, f, K, dK, rho, h, ngb, ngbdist):
    A = 1.0
    N, dim = x.shape
    Nngb = ngb.shape[1]
    a = np.zeros_like(x)
    for i in xrange(N):
        ngbi = ngb[i]
        xngb = x[ngbi]
        mngb = m[ngbi]
        hngb = h[ngbi]
        rngb = ngbdist[i]
        fngb = f[ngbi]
        Ki = K[i]
        dKi = dK[i]
        hi = h[i]
        fi = f[i]
        rhoi = rho[i]
        for j in xrange(1,Nngb):
            atot = mngb[j] * (fi * A * rhoi**(gamma-2.0)) * dKi[j] / hi**4 
            + fngb[j] * A * rho[ngbi[j]]**(gamma-2.0) * DKernel3D(rngb[j]/h[ngbi[j]]) / h[ngbi[j]]**4
            #atot = mngb[j] * (rhoi**(gamma-2.0) + rho[ngbi[j]]**(gamma-2.0)) * dKi[j] / hi**4.0
            for k in xrange(dim):
                dx1 = xngb[j,k] - xngb[0,k]
                dx2 = -np.sign(dx1) * (1.0 - np.abs(dx1))
                if np.abs(dx1) < np.abs(dx2):
                    dx = dx1
                else:
                    dx = dx2
                a[i,k] += atot*dx/rngb[j]
                
    return a
@jit
def KernelSum(f,r,h):
    return np.sum(f*Kernel3D(r/h)/h**3)

In [ ]:
KernelSum(m[ngb[0]], ngbdist[0], h[0])/ rho[0]

In [ ]:
plt.loglog(np.logspace(2,7,5), (392e-6,5e-3,64e-3,0.917,13.7))
plt.loglog(np.logspace(2,7,5), 392e-6*np.logspace(2,7,5)*np.log10(np.logspace(2,7,5)/100))
plt.show()

In [ ]:
optimize.curve_fit(lambda x, a, b: a*x + b, np.linspace(2,7,5), np.log10((392e-6,5e-3,64e-3,0.917,13.7)))

In [ ]:
plt.plot(np.linspace(0,1,100), DKernel3D(np.linspace(0,1,100))); plt.show()

In [ ]:
(10**6/10**2)**2